In [1]:
# !uv pip install -U geemap
# !pip install geopandas
# !pip install earthengine-api

# Water Quality Index Analysis
This is a water quality index analysis of Ganga River.

## Aim
The main aim is to extract the water body thorugh NDWI, MNDWI, AEWI techniques.
We will be using Sentinal-2 imagery for our analysis purpose.

In [2]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
# from shapely.geometry import Polygon, Point
import seaborn as sns
import geemap.colormaps as cm
import geemap.chart as chart
# import pandas as pd
# import pycrs

In [3]:
# Authentication and Intialization of earth engine
try:
    ee.Authenticate()
    ee.Initialize(project='ee-workspacesohan')
except Exception as e:
    print("Successfully authenticated and initialized Earth Engine.")
    print("Error during authentication or initialization:", e)

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


# Steps of the study

1. Kml file to SHP file format
2. Load the file and visualize the study area
3. Satellite data extraction -

    a. NIR

    b. Standard RGB

4. Detection and Extraction of water body
    
    a. Water Body detection through - NDWI, MNDWI, AEWI

    b. Combine the results for optimum  water body selelction
    
    c. Masking out the water body from the correspondend satellite images.

5. Water Quality Index Parametrization.

    a. Salinity

    b. NDTI

    c. Temperature

    d. TDS

6. Statistical Analysis

    a. Zonal Analysis

    b. Point Sampling*

7. Results



> **Note** - just run the follwoing command once!

In [4]:
# # KML to shp file for upload purposes

# input_kml = "/content/drive/MyDrive/6_1-Personal_Project_and_Workspace/2-Projects/2-SiDCREW_Project/1-Study_Area/02-SA-Various River Types/06-Sreerampur_Ganga_River.kml"

# output_shp = "/content/drive/MyDrive/6_1-Personal_Project_and_Workspace/2-Projects/2-SiDCREW_Project/1-Study_Area/02-SA-Various River Types/01-shp-Ganga/06-Sreerampur_Ganga.shp"

# # --- 2. Read the KML file into a GeoDataFrame ---
# # Geopandas will automatically handle different layers within the KML.
# print(f"Reading KML file: {input_kml}")
# gdf = gpd.read_file(input_kml, driver='KML')

# # --- 3. Save the GeoDataFrame to a Shapefile ---
# # The Coordinate Reference System (CRS) is automatically handled.
# # KML is typically WGS84 (EPSG:4326).
# print(f"Saving Shapefile to: {output_shp}")
# gdf.to_file(output_shp, driver='ESRI Shapefile')

# print("\nConversion complete!")



> **Note:** Now that the shp file has been generated I don't know how to upload the shp into an asset in GEE through the python api. So, manually uploading the shp file has to be done.





In [5]:
# Preparing a Map
Map = geemap.Map()
Map.add_basemap('HYBRID')

# Visualization of the Study Area
study_area_ganga = ee.FeatureCollection("projects/ee-workspacesohan/assets/06-Sreerampur_Ganga")

# Projecting Study area on the map
Map.centerObject(study_area_ganga, 10)
Map.addLayer(study_area_ganga, {"color": "ff0000ff", "width": 3, "lineType": "dashed", "fillColor": "00000000"}, "Study Area Ganga")

# Image acquisition and seasonal variation
Now that we have described our study area, the satellite imagery has to be acruired first.

For this we will be using Sentinal-2 L2 satellite data (S2-Surface Reflectance) data provided by the earth engine. The reason is for the already atmospheric and radiometric corrected.

The daterange is a particular object for our study. We will be using the following convention for our study

1. Premonsoon - March, April, and May

2. Monsoon - June, July, August, and September

3. Postmonsoon - October, November

4. Winter - December, January, and February

Therefore we will be making the composite images for the corresponding seasonal changes from the satellite imagery.

In [6]:
# Cloud Masking
# Cloud Masking Function for Sentinel-2
def maskS2clouds(image):
    qa = image.select("QA60")
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Visualization parameters for specific stretch

# --- Defining the reusable function for False Colour Composite Images ---
def get_98_stretch_params(image, geometry, scale=10, gamma=1.3):
    """
    Calculates the 2nd and 98th percentile for an image's NIR bands
    and returns a visualization parameters dictionary.

    Args:
        image (ee.Image): The Earth Engine image to visualize.

    Returns:
        dict: A dictionary of visualization parameters for Map.addLayer().
    """
    false_bands = ["B8", "B4", "B3"]  # NIR, Red, Green

    # Calculate percentiles for the specified geometry
    percentiles = (
        image.select(false_bands)
        .reduceRegion(
            reducer=ee.Reducer.percentile([2, 98]),
            geometry=geometry,
            scale=10,
            maxPixels=1e9,
        )
        .getInfo()
    )  # Fetch the results from the server

    # Dynamically create the min/max lists from the results
    vis_min = [percentiles.get(f"{b}_p2") for b in false_bands if percentiles.get(f"{b}_p2") is not None]
    vis_max = [percentiles.get(f"{b}_p98") for b in false_bands if percentiles.get(f"{b}_p98") is not None]

    # Ensure min and max lists have the same length as bands
    if len(vis_min) != len(false_bands) or len(vis_max) != len(false_bands):
      print("Warning: Percentile calculation failed for some bands.")
      return None # Or return a default parameter

    return {"min": vis_min, "max": vis_max, "bands": false_bands}

# --- Define the resuable function for True Color Composite Images ---
def get_90_stretch_params(image, geometry, scale=10, gamma=1.3):
    """
    Calculates the 10th and 90th percentile for an image's RGB bands
    and returns a visualization parameters dictionary.

    Args:
        image (ee.Image): The Earth Engine image to visualize.

    Returns:
        dict: A dictionary of visualization parameters for Map.addLayer().
    """
    true_bands = ["B4", "B3", "B2"]  # Red, Green, Blue

    # Calculate percentiles for the specified geometry
    percentiles = (
        image.select(true_bands)
        .reduceRegion(
            reducer=ee.Reducer.percentile([10, 90]),
            geometry=geometry,
            scale=10,
            maxPixels=1e9,
        )
        .getInfo()
    )  # Fetch the results from the server

    # Dynamically create the min/max lists from the results
    vis_min = [percentiles.get(f"{b}_p10") for b in true_bands if percentiles.get(f"{b}_p10") is not None]
    vis_max = [percentiles.get(f"{b}_p90") for b in true_bands if percentiles.get(f"{b}_p90") is not None]

    # Ensure min and max lists have the same length as bands
    if len(vis_min) != len(true_bands) or len(vis_max) != len(true_bands):
      print("Warning: Percentile calculation failed for some bands.")
      return None # Or return a default parameter


    return {"min": vis_min, "max": vis_max, "bands": true_bands}

In [7]:
# Filtering the satellite images -- Year Range: 2023 and 2019
'''
The median is often preferred for image composites
as it is less sensitive to outliers (like remaining clouds or other transient features) than the mean.
'''

# Premonsoon range: 2023-10-01 <-- --> 2023-11-30
pre_monsoon_2023 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2023-10-01", "2023-11-30")
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 9))
    .filterBounds(study_area_ganga)
    .map(maskS2clouds)
)
pre_monsoon_2023_composite = pre_monsoon_2023.median().clip(study_area_ganga)

# Premonsoon range: 2019-10-01 <-- --> 2019-11-30
pre_monsoon_2019 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate("2019-10-01", "2019-11-30")
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 9))
    .filterBounds(study_area_ganga)
    .map(maskS2clouds)
)
pre_monsoon_2019_composite = pre_monsoon_2019.median().clip(study_area_ganga)

In [8]:
# Applying dynamic Visualization Parameters for the Premonsoon Composite Images

# Year: 2023
# False Color Composite
pre_monsoon_2023_false_color_vis_params = get_98_stretch_params(
    pre_monsoon_2023_composite, study_area_ganga
)

# True Color Composite
pre_monsoon_2023_true_color_vis_params = get_90_stretch_params(
    pre_monsoon_2023_composite, study_area_ganga
)

# Year: 2019
pre_monsoon_2019_false_color_vis_params = get_98_stretch_params(
    pre_monsoon_2019_composite, study_area_ganga
)

# True Color Composite
pre_monsoon_2019_true_color_vis_params = get_90_stretch_params(
    pre_monsoon_2019_composite, study_area_ganga
)

In [9]:
# Visualization of the Image Composite
# Year: 2023
Map.addLayer(
    pre_monsoon_2023_composite,
    pre_monsoon_2023_true_color_vis_params,
    "Premonsoon 2023 True Color Composite",
)
Map.addLayer(
    pre_monsoon_2023_composite,
    pre_monsoon_2023_false_color_vis_params,
    "Premonsoon 2023 False Color Composite",
)

# Year: 2019
Map.addLayer(
    pre_monsoon_2019_composite,
    pre_monsoon_2019_true_color_vis_params,
    "Premonsoon 2019 True Color Composite",
)
Map.addLayer(
    pre_monsoon_2019_composite,
    pre_monsoon_2019_false_color_vis_params,
    "Premonsoon 2019 False Color Composite",
)
Map.centerObject(study_area_ganga,12)


# Water Body Extraction

This section of will showcase the water body detection capability of the different water body detection indexes and if there is any advantage of using combined algorithms as well.

1. NDWI (_Normalized Difference Water Index_) --


$$
    \text{NDWI} = \frac{Green - NIR}{Green + NIR}
$$

2. MNDWI (_Modified NDWI_) --

$$
    \text{MNDWI} = \frac{Green - SWIR1}{Green + SWIR1}
$$

3. AWEI (_Automated Water Extraction Index_) -- Now there is two algorithms based on the fact that water body with now shadows and water body with shadows

    a. AWEIsh - for shadow regions
    
    $\text{AWEI}_{sh} = Blue + (2.5 × Green) − {1.5 × (NIR + SWIR1)} − (0.25 × SWIR2)$

    b. AWEInsh - for no shadow regions
    
    $\text{AWEI}_{nsh} = 4 × (Green − SWIR1) − (0.25 × NIR + 2.75 × SWIR1)$


> **This is a dummy code for the pansharpening of the bads B11 and B12 as they are 20m resolution.**

# Pansharpening of the bands for better water masking - Advance MNDWI algorithm
The thought is to chnage the band resolution to 10m for the masking purpose. For that we are using B8 (NIR) band as a sharpening band and B3 as a dummy Blue band for HSV conversion.

Walking Through the Script's Logic
Now let's look at the key lines of the function and why they are there.

1. Creating the "Blurry Painting"

```python
coarse_bands = image.select(['B12', 'B11', 'B2'])
```
Why? To use the HSV transform, we need a three-band RGB image. We assign the two SWIR bands we want to sharpen (B12 and B11) to the Red and Green channels. We need a third band for the Blue channel, so we just grab another 20m band (like B2) as a placeholder. This creates our "blurry color painting."

2. Separating Color from Detail

```python
hsv = coarse_bands.rgbToHsv()
```
Why? This is the core step. We convert our "blurry painting" from the RGB model to the HSV model. This gives us three separate layers: Hue (the blurry color), Saturation (the blurry color intensity), and Value (the blurry detail).

3. Swapping the Detail Layer

```python
pan = image.select('B8') # This is our "sharp pencil sketch"
sharpened_hsv = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), pan])
```
Why? This is the "magic" of the fusion.

We take the original hue and saturation bands (preserving the original, correct color information).

We throw away the blurry value band from the original hsv image.

We replace it with our sharp, high-resolution pan image (the 10m NIR band).

We now have a new HSV image with blurry colors but sharp detail.

4. Rebuilding the Final Image

```python
sharpened_rgb = sharpened_hsv.hsvToRgb()
```
Why? We convert the new, hybrid HSV image back into the standard RGB color model for display and analysis. The output is a single, three-band image.

The Red channel of this new image is the sharpened B12, and the Green channel is the sharpened B11. They now have the spatial detail of the 10m NIR band while retaining their original SWIR color properties. That's why the final steps rename the bands to B12_sharp and B11_sharp for clarity.

It might be helpful but I have to check it otherwise.


```python
def sharpen_s2_swir(image):
    """
    Pansharpens the 20m SWIR bands of a Sentinel-2 image to 10m.

    Args:
        image (ee.Image): A Sentinel-2 image with at least B8, B11, and B12.

    Returns:
        ee.Image: The original image with sharpened 'B11_sharp' and 'B12_sharp' bands added.
    """
    # Select the high-resolution band to use for sharpening (NIR is a good choice)
    pan = image.select('B8') # 10m

    # Select the 20m bands to be sharpened.
    # We add a dummy Blue band (B2) to create a 3-band image for HSV conversion.
    coarse_bands = image.select(['B12', 'B11', 'B2']) # SWIR2, SWIR1, Blue

    # Convert the coarse image to HSV color space.
    hsv = coarse_bands.rgbToHsv()
    hue = hsv.select('hue')
    saturation = hsv.select('saturation')

    # Swap the coarse 'value' band with the high-resolution panchromatic band.
    # The result is a sharpened HSV image.
    sharpened_hsv = ee.Image.cat([hue, saturation, pan]).hsvToRgb()

    # Rename the sharpened bands to avoid confusion
    sharpened_bands = sharpened_hsv.rename(['B12_sharp', 'B11_sharp', 'B2_ignore'])

    # Add the newly sharpened 10m SWIR bands to the original image
    return image.addBands(sharpened_bands.select(['B12_sharp', 'B11_sharp']))


# --- Example Usage ---

# Load a Sentinel-2 image
s2_image = ee.Image('COPERUS/S2_SR_HARMONIZED/20240415T044031_20240415T044031_T45RVP')

# Run the sharpening function
sharpened_image = sharpen_s2_swir(s2_image)

# Now calculate MNDWI using the new sharpened B11 band
mndwi_sharpened = sharpened_image.expression(
    '(GREEN - SWIR1_SHARP) / (GREEN + SWIR1_SHARP)', {
        'GREEN': sharpened_image.select('B3'),
        'SWIR1_SHARP': sharpened_image.select('B11_sharp')
    }
).rename('MNDWI_sharp')

# --- Visualization to Compare ---
Map = geemap.Map()
Map.centerObject(s2_image, 13)

# Original MNDWI (GEE automatic resampling)
mndwi_original = s2_image.normalizedDifference(['B3', 'B11']).rename('MNDWI_original')
Map.addLayer(mndwi_original, {'min': -0.5, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Original MNDWI (Blocky)')

# Sharpened MNDWI
Map.addLayer(mndwi_sharpened, {'min': -0.5, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Sharpened MNDWI (10m)')

Map
```
> **Note** - This is a common practice for the visualization of the MNDWI. The earth engine resamples the B11 and B12 by the _nearest neighbour_ algorithm but _pan sharpening_ the bands also might give the good result.


In [10]:
# Pansharpening of the Coarser Bands
def sharpen_s2_swir(image):
    """
    Pansharpens the 20m SWIR bands of a Sentinel-2 image to 10m.

    Args:
        image (ee.Image): A Sentinel-2 image with at least B8, B11, and B12.

    Returns:
        ee.Image: The original image with sharpened 'B11_sharp' and 'B12_sharp' bands added.
    """
    # Select the high-resolution band to use for sharpening (NIR is a good choice)
    pan = image.select('B8') # 10m

    # Select the 20m bands to be sharpened.
    # We add a dummy Blue band (B2) to create a 3-band image for HSV conversion.
    coarse_bands = image.select(['B12', 'B11', 'B2']) # SWIR2, SWIR1, Blue

    # Convert the coarse image to HSV color space.
    hsv = coarse_bands.rgbToHsv()
    hue = hsv.select('hue')
    saturation = hsv.select('saturation')

    # Swap the coarse 'value' band with the high-resolution panchromatic band.
    # The result is a sharpened HSV image.
    sharpened_hsv = ee.Image.cat([hue, saturation, pan]).hsvToRgb()

    # Rename the sharpened bands to avoid confusion
    sharpened_bands = sharpened_hsv.rename(['B12_sharp', 'B11_sharp', 'B2_ignore'])

    # Add the newly sharpened 10m SWIR bands to the original image
    return image.addBands(sharpened_bands.select(['B12_sharp', 'B11_sharp']))



In [11]:
# Sharpening of the Composite Images
s2_image_premonsoon_2023 = pre_monsoon_2023_composite
s2_image_premonsoon_2019 = pre_monsoon_2019_composite

sharpened_image_premonsoon_2023 = sharpen_s2_swir(s2_image_premonsoon_2023)
sharpened_image_premonsoon_2019 = sharpen_s2_swir(s2_image_premonsoon_2019)

In [12]:
# Water Body Extraction

#NDWI
ndwi_premonsoon_2023 = pre_monsoon_2023_composite.normalizedDifference(["B3", "B8"]).rename("NDWI")
ndwi_premonsoon_2019 = pre_monsoon_2019_composite.normalizedDifference(["B3", "B8"]).rename("NDWI")

#MNDWI
mndwi_sharpened_premonsoon_2023 = sharpened_image_premonsoon_2023.expression(
    '(GREEN - SWIR1_SHARP) / (GREEN + SWIR1_SHARP)', {
        'GREEN': sharpened_image_premonsoon_2023.select('B3'),
        'SWIR1_SHARP': sharpened_image_premonsoon_2023.select('B11_sharp')
    }
).rename('MNDWI_sharp_premonsoon_2023')

mndwi_sharpened_premonsoon_2019 = sharpened_image_premonsoon_2019.expression(
    '(GREEN - SWIR1_SHARP) / (GREEN + SWIR1_SHARP)', {
        'GREEN': sharpened_image_premonsoon_2019.select('B3'),
        'SWIR1_SHARP': sharpened_image_premonsoon_2019.select('B11_sharp')
    }
).rename('MNDWI_sharp_premonsoon_2019')

# print(mndwi_sharpened_premonsoon_2023.getInfo())

# AWEI-shadows
awei_sh_premonsoon_2023 = pre_monsoon_2023_composite.expression(
    'Blue+(2.5*Green)-(1.5*(NIR+SWIR1))-(0.25*SWIR2)',
    {
        'Blue': pre_monsoon_2023_composite.select('B2'),
        'Green': pre_monsoon_2023_composite.select('B3'),
        'NIR': pre_monsoon_2023_composite.select('B8'),
        'SWIR1': sharpened_image_premonsoon_2023.select('B11_sharp'),
        'SWIR2': sharpened_image_premonsoon_2023.select('B12_sharp')
    }
)

awei_sh_premonsoon_2019 = pre_monsoon_2019_composite.expression(
    'Blue+(2.5*Green)-(1.5*(NIR+SWIR1))-(0.2*SWIR2)',
    {
        'Blue': pre_monsoon_2019_composite.select('B2'),
        'Green': pre_monsoon_2019_composite.select('B3'),
        'NIR': pre_monsoon_2019_composite.select('B8'),
        'SWIR1': sharpened_image_premonsoon_2019.select('B11_sharp'),
        'SWIR2': sharpened_image_premonsoon_2019.select('B12_sharp')
    }
)

# AWEI-noshadows
awei_nsh_premonsoon_2023 = pre_monsoon_2023_composite.expression(
    '4*(Green-SWIR1)-(0.25*NIR+2.75*SWIR1)',
    {
        'Green': pre_monsoon_2023_composite.select('B2'),
        'SWIR1': sharpened_image_premonsoon_2023.select('B11_sharp'),
        'NIR': pre_monsoon_2023_composite.select('B8')
    }
)

awei_nsh_premonsoon_2019 = pre_monsoon_2019_composite.expression(
    '4*(Green-SWIR1)-(0.25*NIR+2.75*SWIR1)',
    {
        'Green': pre_monsoon_2019_composite.select('B2'),
        'SWIR1': sharpened_image_premonsoon_2019.select('B11_sharp'),
        'NIR': pre_monsoon_2019_composite.select('B8')
    }
)

# print(awei_premonsoon_2023.getInfo())

In [13]:
# # Visualize
# Map.addLayer(ndwi_premonsoon_2023, {'min': -0.2, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'NDWI premonsoon 2023')
# Map.addLayer(mndwi_sharpened_premonsoon_2023, {'min': -0.2, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'MNDWI premonsoon 2019')
# Map.addLayer(awei_sh_premonsoon_2023, {'min': -0.2, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'AWEIsh premonsoon 2023')
# Map.addLayer(awei_nsh_premonsoon_2023, {'min': -0.2, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'AWEInsh premonsoon 2023')
# Map.centerObject(study_area_ganga, 12)
# Map

In [14]:
# Thresholding the Water Body
# Year: 2023
ndwi_premonsoon_2023_mask = ndwi_premonsoon_2023.gte(0.05)
mndwi_sharpened_premonsoon_2023_mask = mndwi_sharpened_premonsoon_2023.gte(0.05)
awei_sh_premonsoon_2023_mask = awei_sh_premonsoon_2023.gte(0.05)
awei_nsh_premonsoon_2023_mask = awei_nsh_premonsoon_2023.gte(0.05)
comb_premonsoon_2023_water_mask = ndwi_premonsoon_2023_mask.add(mndwi_sharpened_premonsoon_2023_mask).add(awei_sh_premonsoon_2023_mask).add(awei_nsh_premonsoon_2023_mask).rename("water_mask_premonsoon_2023")

# Year: 2019
ndwi_premonsoon_2019_mask = ndwi_premonsoon_2019.gte(0.05)
mndwi_sharpened_premonsoon_2019_mask = mndwi_sharpened_premonsoon_2019.gte(0.05)
awei_sh_premonsoon_2019_mask = awei_sh_premonsoon_2019.gte(0.05)
awei_nsh_premonsoon_2019_mask = awei_nsh_premonsoon_2019.gte(0.05)
comb_premonsoon_2019_water_mask = ndwi_premonsoon_2019_mask.add(mndwi_sharpened_premonsoon_2019_mask).add(awei_sh_premonsoon_2019_mask).add(awei_nsh_premonsoon_2019_mask).rename("water_mask_premonsoon_2019")

# # Removing small patches
# # comb_premonsoon_2023_water_mask_foc = comb_premonsoon_2023_water_mask.focal_max(1, "square", "pixels",9).focal_min(1, "square", "pixels",9)

# # Visualize
# Map.addLayer(ndwi_premonsoon_2023_mask,{}, "NDWI premonsoon 2023")
# Map.addLayer(mndwi_sharpened_premonsoon_2023_mask,{}, "MNDWI premonsoon 2023")
# Map.addLayer(awei_sh_premonsoon_2023_mask,{}, "AWEIsh premonsoon 2023")
# Map.addLayer(awei_nsh_premonsoon_2023_mask,{}, "AWEInsh premonsoon 2023")
# Map.addLayer(comb_premonsoon_2023_water_mask,{}, "Water Mask Premonsoon 2023")
# # Map.addLayer(comb_premonsoon_2023_water_mask_foc,{}, "Water Mask Premonsoon 2023 Foc")
# Map.centerObject(study_area_ganga, 12)
# Map

In [15]:
# Masking the water body from the original fcc image through combined mask

# # Masking the 2023 pre-monsoon false color composite
pre_monsoon_2023_masked_water = pre_monsoon_2023_composite.updateMask(comb_premonsoon_2023_water_mask)
pre_monsoon_2019_masked_water = pre_monsoon_2019_composite.updateMask(comb_premonsoon_2019_water_mask)

# # Masking the 2019 pre-monsoon false color composite
# pre_monsoon_2019_masked = pre_monsoon_2019_composite.updateMask(comb_premonsoon_2023_water_mask) # Use 2023 mask for comparison

# # Visualization Parameters
# vis = {
#     "min": 0.0,
#     "max": 1
# }

# # Visualize the masked images (optional)
Map.addLayer(pre_monsoon_2023_masked_water, {}, "Premonsoon 2023 Water Masked")
Map.addLayer(pre_monsoon_2019_masked_water, {}, "Premonsoon 2019 Masked FCC")
Map.centerObject(study_area_ganga, 12)
Map

Map(center=[22.385600404959956, 88.22246115951589], controls=(WidgetControl(options=['position', 'transparent_…

### Calculation of NDTI (Normalized Difference Turbidity Index)
The NDTI defines the turbidity of water bodies. It is calculated using the following formula:
$$ NDTI = \frac{(B4 - B3)}{(B4 + B3)} $$
where,
- \(B4\) is the Red band
- \(B3\) is the Green band  

### Calculation of TDS (Total Dissolved Solids)
The TDS is calculated using the following empirical formula:
$$
TDS = \frac{B5-B2}{B5+b2}
$$
where,
- \(B4\) is the Red band at 10m resolution
- \(B5\) is the Red Edge 1 band at 20m resolution. Resmaple to 10m by the earth engine.

We can pan-sharpen the B5 band but for analysing purpose that is refrained from the scope of this study.

$$
    TDS1 = B5+B4+B3
$$

$$
    TDS2 = B5+B4
$$

### Calculation of the Salinity of the Water
The salinity is calculated using the following emperical formula:
$$
    \text{Salinity} = \sqrt{B3^{2} \times B8^{2}}
$$



In [16]:
# NDTI calculation
premonsoon_2023_ndti = pre_monsoon_2023_masked_water.normalizedDifference(["B4", "B3"]).rename("NDTI")
premonsoon_2019_ndti = pre_monsoon_2019_masked_water.normalizedDifference(["B4", "B3"]).rename("NDTI")

# # TDS calculation
premonsoon_2023_tds = pre_monsoon_2023_masked_water.normalizedDifference(["B5", "B2"]).rename("TDS")
premonsoon_2023_tds_1 = pre_monsoon_2023_masked_water.expression(
    'B5+B4+B3',
    {
        'B5': pre_monsoon_2023_composite.select('B5'),
        'B4': pre_monsoon_2023_composite.select('B4'),
        'B3': pre_monsoon_2023_composite.select('B3')
    }
).rename("TDS_1")
premonsoon_2023_tds_2 = pre_monsoon_2023_masked_water.expression(
    'B5+B4',
    {
        'B5': pre_monsoon_2023_composite.select('B5'),
        'B4': pre_monsoon_2023_composite.select('B4')
    }
).rename("TDS_2")

premonsoon_2019_tds = pre_monsoon_2019_masked_water.normalizedDifference(["B5", "B2"]).rename("TDS")
premonsoon_2019_tds_1 = pre_monsoon_2019_masked_water.expression(
    'B5+B4+B3',
    {
        'B5': pre_monsoon_2019_masked_water.select('B5'),
        'B4': pre_monsoon_2019_masked_water.select('B4'),
        'B3': pre_monsoon_2019_masked_water.select('B3')
    }
).rename("TDS_1")
premonsoon_2019_tds_2 = pre_monsoon_2019_masked_water.expression(
    'B5+B4',
    {
        'B5': pre_monsoon_2019_masked_water.select('B5'),
        'B4': pre_monsoon_2019_masked_water.select('B4')
    }
).rename("TDS_2")

# Salinity
premonsoon_2023_salinity = pre_monsoon_2023_masked_water.expression(
    'sqrt(pow(B3,2)*pow(B8,2))',
    {
        'B3': pre_monsoon_2023_masked_water.select('B3'),
        'B8': pre_monsoon_2023_masked_water.select('B8')
    }
).rename("Salinity")
premonsoon_2023_salinity_1 = pre_monsoon_2023_masked_water.expression(
    'sqrt(B4 * B2)',
    {
        'B4': pre_monsoon_2023_masked_water.select('B4'),
        'B2': pre_monsoon_2023_masked_water.select('B2')
    }
).rename("Salinity_1")

premonsoon_2019_salinity = pre_monsoon_2019_masked_water.expression(
    'sqrt(pow(B3,2)*pow(B8,2))',
    {
        'B3': pre_monsoon_2019_masked_water.select('B3'),
        'B8': pre_monsoon_2019_masked_water.select('B8')
    }
).rename("Salinity")
premonsoon_2019_salinity_1 = pre_monsoon_2019_masked_water.expression(
    'sqrt(B4 * B2)',
    {
        'B4': pre_monsoon_2019_masked_water.select('B4'),
        'B2': pre_monsoon_2019_masked_water.select('B2')
    }
).rename("Salinity_1")

In [17]:
# # Visualization
# Map.addLayer(premonsoon_2023_ndti, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 NDTI')
# Map.addLayer(premonsoon_2023_tds, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 TDS')
# Map.addLayer(premonsoon_2023_tds_1, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 TDS-1')
# Map.addLayer(premonsoon_2023_tds_2, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 TDS-2')
# Map.addLayer(premonsoon_2023_salinity, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 Salinity')
# Map.addLayer(premonsoon_2023_salinity_1, {'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}, 'Premonsoon 2023 Salinity-1')
# Map.centerObject(study_area_ganga, 11)
# Map

In [18]:
# Exporting the variables
# For Composite
task_19_Sreerampur = ee.batch.Export.image.toDrive(pre_monsoon_2019_composite,
                                     folder='GEE_Exports', description='Postmonsoon 2019 Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(pre_monsoon_2023_composite,
                                     folder='GEE_Exports', description='Postmonsoon 2023 Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# For NDWI Mask
task_19_Sreerampur = ee.batch.Export.image.toDrive(ndwi_premonsoon_2019_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2019 Water Mask Sreerampur NDWI',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(ndwi_premonsoon_2023_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2023 Water Mask Sreerampur NDWI',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# MNDWI
task_19_Sreerampur = ee.batch.Export.image.toDrive(mndwi_sharpened_premonsoon_2019_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2019 Water Mask Sreerampur MNDWI',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(mndwi_sharpened_premonsoon_2023_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2023 Water Mask Sreerampur MNDWI',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# MNDWI
task_19_Sreerampur = ee.batch.Export.image.toDrive(comb_premonsoon_2019_water_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2019 Combined Water Mask Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(comb_premonsoon_2023_water_mask,
                                     folder='GEE_Exports', description='Postmonsoon 2023 Combined Water Mask Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# For NDTI
task_19_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2019_ndti,
                                     folder='GEE_Exports', description='Postmonsoon 2019 NDTI Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2023_ndti,
                                     folder='GEE_Exports', description='Postmonsoon 2023 NDTI Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# For TDS1
task_19_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2019_tds_1,
                                     folder='GEE_Exports', description='Postmonsoon 2019 TDS-1 Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2023_tds_1,
                                     folder='GEE_Exports', description='Postmonsoon 2023 TDS-1 Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()

# Salinity
task_19_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2019_salinity,
                                     folder='GEE_Exports', description='Postmonsoon 2019 Salinity Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_19_Sreerampur.start()

task_23_Sreerampur = ee.batch.Export.image.toDrive(premonsoon_2023_salinity,
                                     folder='GEE_Exports', description='Postmonsoon 2023 Salinity Sreerampur',
                                     scale=10,
                                     region=study_area_ganga.geometry(), formatOptions={'cloudOptimized': True})

# Start the export task
task_23_Sreerampur.start()